<center><h1  style="color:white; background-color:#000000; border-radius: 0px; padding:25px;"> MGS-GRF </h1></center>

This notebook illustrates the way of uysing our propposed method with an example.

In [ ]:
import os
import sys
sys.path.insert(1, os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
from pathlib import Path
from collections import Counter

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import OneHotEncoder

from validation.classif_experiments import read_subsampling_indices, subsample_to_ratio_indices
from data.data import load_BankChurners_data
import lightgbm as lgb

In [ ]:
from mgs_grf.over_sampling import MGSGRFOverSampler
from mgs_grf.forest_for_categorical import DrfSk

<center><h1  style="color:white; background-color:#808b96; border-radius: 10px; padding:15px;"> BankChurners data </h1></center>

In [ ]:
output_dir_path = "../saved_experiments_categorial_features/BankChurners_example"
Path(output_dir_path).mkdir(parents=True, exist_ok=True)

In [ ]:
## Import of the original data
initial_X,initial_y = load_BankChurners_data()
numeric_features = [0,2,7,8,9,10,11,12,13,14,15,16,17,18]
categorical_features = [1,3,4,5,6]

## Decreasing the imbalance ratio from 20% to 1%.
indices_kept_1 = subsample_to_ratio_indices(X=initial_X,y=initial_y,ratio=0.01,seed_sub=5,output_dir_subsampling=output_dir_path,
                                            name_subsampling_file='BankChurners_sub_original_to_1')
X, y = initial_X[indices_kept_1],initial_y[indices_kept_1]
print("Bankchurners 1% imbalance ratio : ", Counter(y))

## Splitting the data intro train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
## Apply MGS-GRF procedure to oversample the data
mgs_grf = MGSGRFOverSampler(K=len(numeric_features),categorical_features=categorical_features, Classifier=DrfSk(),random_state=0)
balanced_X, balanced_y = mgs_grf.fit_resample(X_train,y_train)
print("Augmented data : ", Counter(balanced_y))

## Encode the categorical variables
enc = OneHotEncoder(handle_unknown='ignore',sparse_output=False)
balanced_X_encoded = enc.fit_transform(balanced_X[:,categorical_features])
balanced_X_final = np.hstack((balanced_X[:,numeric_features],balanced_X_encoded))
clf_mgs = lgb.LGBMClassifier(n_estimators=100,verbosity=-1, random_state=0)
# Fit the final classifier on the augmented data
clf_mgs.fit(balanced_X_final, balanced_y)

## Evaluation on test set
X_test_enc = enc.transform(X_test[:,categorical_features])
X_test_final = np.hstack((X_test[:,numeric_features],X_test_enc))
y_pred_probas_mgs_grf = clf_mgs.predict_proba(X_test_final)[:,1]
print("PR AUC induced by MGS-GRF : ", average_precision_score(y_test, y_pred_probas_mgs_grf))

In [ ]:
## Comparison when applying no rebalancing strategy
enc = OneHotEncoder(handle_unknown='ignore',sparse_output=False)
X_train_encoded = enc.fit_transform(X_train[:,categorical_features])
X_train_final = np.hstack((X_train[:,numeric_features],X_train_encoded))
clf_none = lgb.LGBMClassifier(n_estimators=100,verbosity=-1, random_state=0)
clf_none.fit(X_train_final, y_train)

X_test_enc = enc.transform(X_test[:,categorical_features])
X_test_final = np.hstack((X_test[:,numeric_features],X_test_enc))
y_pred_probas_none = clf_none.predict_proba(X_test_final)[:,1]
print("PR AUC induced by applying no rebalncing strategy : ", average_precision_score(y_test, y_pred_probas_none))

<center><h1  style="color:white; background-color:#808b96; border-radius: 10px; padding:15px;"> END </h1></center>